In [1]:
import malaya

/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
prob_corrector = malaya.spell.probability()

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [4]:
import sentencepiece as spm
from prepro_utils import preprocess_text, encode_ids, encode_pieces

sp_model = spm.SentencePieceProcessor()
sp_model.Load('sp10m.cased.v8.model')

with open('sp10m.cased.v8.vocab') as fopen:
    v = fopen.read().split('\n')[:-1]
v = [i.split('\t') for i in v]
v = {i[0]: i[1] for i in v}

class Tokenizer:
    def __init__(self, v):
        self.vocab = v
        pass
    
    def tokenize(self, string):
        return encode_pieces(sp_model, string, return_unicode=False, sample=False)
    
    def convert_tokens_to_ids(self, tokens):
        return [sp_model.PieceToId(piece) for piece in tokens]
    
    def convert_ids_to_tokens(self, ids):
        return [sp_model.IdToPiece(i) for i in ids]
    
tokenizer = Tokenizer(v)

In [5]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling
import numpy as np
import tensorflow as tf

In [6]:
string1 = 'krjaan patut bagi pencen awal skit kepada warga emas supaya emosi lebih stabil'

prob_corrector.correct_text(string1)

'kerjaan patut bagi pencen awal sakit kepada warga emas supaya emosi lebih stabil'

In [7]:
text_mask = string1.replace('skit', '**mask**')

In [8]:
def tokens_to_masked_ids(tokens, mask_ind):
    masked_tokens = tokens[:]
    masked_tokens[mask_ind] = "<mask>"
    masked_tokens = ["<cls>"] + masked_tokens + ["<sep>"]
    masked_ids = tokenizer.convert_tokens_to_ids(masked_tokens)
    return masked_ids

In [9]:
possible_states = prob_corrector.edit_candidates('skit')
possible_states

{'sakit', 'sakita', 'sakiti', 'sekita', 'sikit', 'skit'}

In [10]:
[tokenizer.tokenize(state) for state in possible_states]

[['▁se', 'kita'],
 ['▁sakit', 'i'],
 ['▁sakit', 'a'],
 ['▁sikit'],
 ['▁skit'],
 ['▁sakit']]

In [11]:
bert_config = modeling.BertConfig.from_json_file('BASE_config.json')

In [12]:
class Model:
    def __init__(
        self,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        
        model = modeling.BertModel(
            config=bert_config,
            is_training=False,
            input_ids=self.X,
            use_one_hot_embeddings=False)
        
        output_layer = model.get_sequence_output()
        embedding = model.get_embedding_table()
        
        with tf.variable_scope('cls/predictions'):
            with tf.variable_scope('transform'):
                input_tensor = tf.layers.dense(
                    output_layer,
                    units = bert_config.hidden_size,
                    activation = modeling.get_activation(bert_config.hidden_act),
                    kernel_initializer = modeling.create_initializer(
                        bert_config.initializer_range
                    ),
                )
                input_tensor = modeling.layer_norm(input_tensor)
            
            output_bias = tf.get_variable(
            'output_bias',
            shape = [bert_config.vocab_size],
            initializer = tf.zeros_initializer(),
            )
            logits = tf.matmul(input_tensor, embedding, transpose_b = True)
            self.logits = tf.nn.bias_add(logits, output_bias)

In [13]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()

sess.run(tf.global_variables_initializer())
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'bert')




Instructions for updating:
Use keras.layers.dense instead.


In [14]:
cls = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'cls')
cls

[<tf.Variable 'cls/predictions/transform/dense/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/transform/dense/bias:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/transform/LayerNorm/beta:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/transform/LayerNorm/gamma:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/output_bias:0' shape=(30000,) dtype=float32_ref>]

In [15]:
saver = tf.train.Saver(var_list = var_lists + cls)
saver.restore(sess, 'bert-base-v2/model.ckpt')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from bert-base-v2/model.ckpt


In [16]:
replaced_masks = [text_mask.replace('**mask**', state) for state in possible_states]
replaced_masks

['krjaan patut bagi pencen awal sekita kepada warga emas supaya emosi lebih stabil',
 'krjaan patut bagi pencen awal sakiti kepada warga emas supaya emosi lebih stabil',
 'krjaan patut bagi pencen awal sakita kepada warga emas supaya emosi lebih stabil',
 'krjaan patut bagi pencen awal sikit kepada warga emas supaya emosi lebih stabil',
 'krjaan patut bagi pencen awal skit kepada warga emas supaya emosi lebih stabil',
 'krjaan patut bagi pencen awal sakit kepada warga emas supaya emosi lebih stabil']

In [17]:
def generate_ids(mask):
    tokens = tokenizer.tokenize(mask)
    input_ids = [tokens_to_masked_ids(tokens, i) for i in range(len(tokens))]
    tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
    return tokens, input_ids, tokens_ids

In [18]:
ids = [generate_ids(mask) for mask in replaced_masks]
tokens, input_ids, tokens_ids = list(zip(*ids))

In [19]:
indices, ids = [], []
for i in range(len(input_ids)):
    indices.extend([i] * len(input_ids[i]))
    ids.extend(input_ids[i])

In [20]:
masked_padded = tf.keras.preprocessing.sequence.pad_sequences(ids,padding='post')
masked_padded.shape

(87, 17)

In [21]:
preds = sess.run(tf.nn.log_softmax(model.logits), feed_dict = {model.X: masked_padded})
preds.shape

(87, 17, 30000)

In [22]:
indices = np.array(indices)
scores = []

for i in range(len(tokens)):
    filter_preds = preds[indices == i]
    total = np.sum([filter_preds[k, k + 1, x] for k, x in enumerate(tokens_ids[i])])
    scores.append(total)
    
scores

[-156.8316, -156.38635, -157.00029, -145.90393, -146.15497, -146.45107]

In [23]:
prob_scores = np.array(scores) / np.sum(scores)
prob_scores

array([0.17258362, 0.17209364, 0.17276925, 0.16055839, 0.16083464,
       0.16116047], dtype=float32)

In [24]:
probs = list(zip(possible_states, prob_scores))
probs.sort(key = lambda x: x[1])  
probs

[('sikit', 0.16055839),
 ('skit', 0.16083464),
 ('sakit', 0.16116047),
 ('sakiti', 0.17209364),
 ('sekita', 0.17258362),
 ('sakita', 0.17276925)]